<a href="https://colab.research.google.com/github/pacasarini/web_mining_crawler_scraping/blob/main/projeto_web_mining_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Faculdade Impacta
##Projeto de Web Mining & Crawler Scraping
##Prof. Fernando de Sequeira

##Grupo:
##- Alexandre
##- Arthur Bassetti
##- Luis Thiago Amorim de Moura
##- Paula Cristina Casarini

##Para este projeto escolhemos o banco de dados do eBit: https://www.ebit.com.br/reputacao-de-lojas.
##Nossa finalidade será realizar uma análise de sentimentos de opiniões sobre empresas por meio das avaliações contidas neste banco de dados.





##Etapas:
Entendimento dos Dados<br>
A etapa de entendimento dos dados deve envolver a coleta e a descrição dos dados coletados (formato, tamanho, tipo dos dados, etc.)<br>
<br>
Preparação dos Dados<br>
A preparação dos dados deve envolver a extração do conteúdo textual, processamento linguístico necessário para o problema, e criação do vetor de características.<br>
<br>
Modelagem<br>
O trabalho deverá aplicar um modelo de  nos dados coletados e preparados e aplicar qualquer algoritmo de classificação, agrupamento ou recuperação de informação.<br>
<br>
Avaliação<br>
A etapa de avaliação deve mostrar os resultados dos modelos aplicados, comparando e mostrando as conclusões.<br>


##Importando as Bibliotecas

In [4]:
import requests
import os
import pandas as pd

###Através do menu "Inspecionar" do browser, identificamos que se tratava de uma pagina dinamica em javascript que é alimentada por conteúdo em json, na seguinte url: https://reputation-api.ebit.com.br/reputation/getall?active=true&shuffle=true&medal={tipo} , onde existem quatro tipos (diamante, ouro, prata e bronze), totalizando o total de 458 empresas. Por este motivo não foi possível/necessária a utilização da biblioteca 'BeautifulSoup'.

In [5]:
#separando as empresas por tipos
tipos = ['diamante', 'ouro', 'prata', 'bronze']

dados_dict = []
for tipo in tipos:
  url = f'https://reputation-api.ebit.com.br/reputation/getall?active=true&shuffle=true&medal={tipo}'
  headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36'
  }
  response = requests.get(url, headers=headers)

  dados = response.json()
  print(f'Quantidade de empresas que são {tipo}: ' + str(dados['Count']))

  for x in dados['Data']:
    dados_dict.append(x)


Quantidade de empresas que são diamante: 72
Quantidade de empresas que são ouro: 10
Quantidade de empresas que são prata: 2
Quantidade de empresas que são bronze: 374


In [6]:
dados_dict[0]   # verificando o retorno da coleta para verificar as informações que ela traz

{'Id': 2637,
 'Date': '2023-10-05T00:00:00',
 'Medal': 'Diamante',
 'DeliveryTime': 97.36842105263158,
 'ReturnPurchase': 81.57894736842105,
 'FriendRecommendation': 64.11,
 'CompanyName': 'Zelo',
 'CompanyDescription': 'A Zelo é especializada em artigos para cama, mesa e banho, tendo inclusive uma linha própria de produtos, além das marcas tradicionais do mercado. Na loja virtual você recebe dicas de como montar o seu enxoval e de como lavar adequadamente os produtos.',
 'StoreUrl': 'www.zelo.com.br',
 'EbitUrlName': 'Zelo',
 'CompanyStatus': 'Ativa',
 'QuantitySurveys': 78038,
 'CompanySearch': 'Zelo',
 'MedalId': 0,
 'CompanyType': 'Loja',
 'FlagLgpd': 0}

In [8]:
#Funcao Salvar arquivo: vai pegar o conteudo do json (dicionário) e salvar cada empresa em seu respectivo diretório, separada entre as categorias. Os comentários dos usurarios serao salvos em 3 arquivos separados: elogio.txt, reclamacao.txt e sugestao.txt
def salva_arquivo(pasta, nome_arquivo, conteudo):
  if not os.path.isdir(pasta):
    os.makedirs(pasta, exist_ok=True)
  with open(f'{pasta}/{nome_arquivo}', 'a+', encoding='utf-8') as file:
    file.write(conteudo)

In [9]:
#Criando um DataFrame vazio para receber informações dos Comentarios para analise de Sentimentos
df = pd.DataFrame(columns=[
    'Id',
    'CompanyId',
    'Name',
    'AnswerDate',
    'CommentaryType',
    'Commentary',
    'Likes',
    'Dislikes',
    'CompanyName'
])

In [10]:
#Gerar Arquivos com os comentarios e um DataFrame
for x in dados_dict:
  id = x['Id']
  medal = x['Medal']
  company = x['CompanyName']

  url = 'https://reputation-api.ebit.com.br/reputation/getCommentaries?companyId={}&type=&offset=0&qty=100'.format(id)
  headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36'
  }
  response = requests.get(url, headers=headers)

  dados = response.json()
  for i in dados['Data']:
    comentaryType = i['CommentaryType']
    comentary = i['Commentary']
    caminhoArquivo = medal + '/' + company + '/' + '/'
    salva_arquivo(caminhoArquivo, (comentaryType + '.txt'), (comentary + ' '))
    #Criar um DataFrame temporario e inserir cada json no Dataframe "df"
    i['CompanyName'] = company #O dicionario de Comentarios recebe o campo CompanyName
    temp_df = pd.DataFrame([i])
    df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df.head()